# groupbyrule: deduplicate data using fuzzy and deterministic matching rules

🚧 under construction 🚧

**groupbyrule** is a Python package for data cleaning and data integration. It provides an extension of [pandas](https://pandas.pydata.org/)' [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function which, instead of grouping rows by a given identifier, groups rows based on logical rules and partial matching. In other words, it provides tools for deterministic record linkage and entity resolution in structured databases. It can also be used for *blocking*, a form of filtering used to speed-up more complex entity resolution algorithms. See the references below to learn more about these topics.

One of the main goal of **groupbyrule** is to be *user-friendly*. Matching rules and clustering algorithms are composable, and the performance of algorithms can be readily evaluted given training data. The package is built on top of [pandas](https://pandas.pydata.org) for data manipulation and on [igraph](https://igraph.org/python/) for graph clustering and related computations.

## Installation

🚧

## Examples

Consider the `RLdata500` dataset from the [RecordLinkage R package](https://www.google.com/search?channel=fs&client=ubuntu&q=recordlinkage+r+package).

In [1]:
from groupbyrule import RLdata500

RLdata500.head()

,fname_c1,fname_c2,lname_c1,lname_c2,by,bm,bd
1,CARSTEN,NaN,MEIER,NaN,1949,7,22
2,GERD,NaN,BAUER,NaN,1968,7,27
3,ROBERT,NaN,HARTMANN,NaN,1930,4,30
4,STEFAN,NaN,WOLFF,NaN,1957,9,2
5,RALF,NaN,KRUEGER,NaN,1966,1,13


We deduplicate this dataset by linking records which match either on both first name (`fname_c1`) and last name (`lname_c1`), on both first name and birth day (`bd`), or on both last name and birth day. Linkage transitivity is resolved, by default, by considering connected components of the resulting graph.

In [2]:
from groupbyrule import Any, Match, groupby
import pandas as pd

rule = Any(Match("fname_c1", "lname_c1"),
           Match("fname_c1", "bd"),
           Match("lname_c1", "bd"))

groupby(RLdata500, rule)


Note that this is not the best way to deduplicate this dataset. However, it showcases the composability of matching rules. The specific rules themselves (exact matching, fuzzy string matching, different clustering algorithms to resolve transitivity) can be customized as needed. A more complete overview is available [here]() 🚧.

A better way to deduplicate this data is to link all pairs of records which agree on all but at most one attribute. This is done below, with the precision and recall computed from the ground truth membership vector `identity_RLdata500`.

In [3]:
from groupbyrule import AllButK, precision_recall, identity_RLdata500

RLdata500.pipe(groupby, AllButK("fname_c1", "lname_c1", "bd", "bm", "by", k=1))\
         .pipe(precision_recall, identity_RLdata500)

NameError: name 'precision_recall' is not defined


### Overview of Linkage Rules

🚧

### Overview of Clustering Algorithms

🚧

## References

- Binette, O. & Steorts, R.C. (2021) Almost All of Statistical Entity Resolution. arXiv e-prints, arxiv:
-  